## Air Quality in Dar es Salaam Tz

In [2]:
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pytz
from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg
from pprint import PrettyPrinter
import plotly.express as px
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA

#### Prepare Data

Connect to MongoDB

In [ ]:
client = MongoClient(host="localhost", port=27017)
db = client["air-quality"]
dar = db["dar-es-salaam"]

In [ ]:
for c in db.list_collections():
    print(c["name"])

Explore

In [ ]:
pp = PrettyPrinter(indent=2)
result = dar.find_one({})
pp.pprint(result)

What are the sites in the data?

In [ ]:
sites = dar.distinct("metadata.site")
sites

Which sites has the largest document counts?

In [ ]:
result = [{'_id': 23, 'count': dar.count_documents({"metadata.site": 23})}, {'_id': 11, 'count':  dar.count_documents({"metadata.site": 11})}]
readings_per_site = list(result)
readings_per_site

#### Import Data with the wrangle function

In [3]:
def wrangle(collection):
    results = collection.find(
        {"metadata.site": 11, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )

    y = pd.DataFrame(results).set_index("timestamp")
    
    # Localize time zone
    y.index= y.index.tz_localize("UTC").tz_convert("Africa/Dar_es_Salaam")
    # Remove Outlines
    y = y[y["P2"] < 100]
    
    # Resample
    y = y["P2"].resample("1H").mean().fillna(method="ffill").to_frame()
    y = pd.Series(y["P2"])
    return y

Explore: Time Series Plots

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y.plot(ax=ax, xlabel="Date",ylabel="PM2.5 Level", title="Dar es Salaam PM2.5 Lveles")

Rolling Averages plot:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y.rolling(168).mean().plot(ax=ax,xlabel="Date", ylabel="PM2.5 Level", title="Dar es Salaam PM2.5 Levels, 7-Day Rolling Average")

ACF Plot

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.title("Dar es Salaam PM2.5 Readings, ACF")

PACF PLot

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_pacf(y, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.title("Dar es Salaam PM2.5 Readings, PACF")

Split into Test and Train Sets

In [ ]:
cutoff_test = int(len(y) * .9)
y_train = y.iloc[:cutoff_test]
y_test = y.iloc[cutoff_test:]
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

#### Build Model

Base Model

In [ ]:
y_train_mean = y_train.mean()
y_pred_baseline = [y_train_mean] * len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

Iterate: Train Autoreg model

In [ ]:
# Create range to test different lags
p_params = range(1, 31)

# Create empty list to hold mean absolute error scores
maes = []

# Iterate through all values of p in `p_params`
for p in p_params:
    # Build model
    model = AutoReg(y_train, lags=p).fit()

    # Make predictions on training data, dropping null values caused by lag
    y_pred = model.predict().dropna()

    # Calculate mean absolute error for training data vs predictions
    mae = mean_absolute_error(y_train.iloc[p:], y_pred)

    # Append `mae` to list `maes`
    maes.append(mae)

# Put list `maes` into Series with index `p_params`
mae_series = pd.Series(maes, name="mae", index=p_params)

# Inspect head of Series
mae_series.head()

The best value of p:

In [ ]:
best_p = np.argmin(maes)
best_model = AutoReg(y_train, lags=best_p).fit()

Predictions:

In [ ]:
y_train_resid = y_train - y_pred
y_train_resid.name = "residuals"
y_train_resid.head()

Residual Plot

In [ ]:
# Plot histogram of residuals
y_train_resid.hist()
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.title("Best Model, Training Residuals")

ACF for the Residuals

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y_train_resid, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.title("Dar es Salaam, Training Residuals ACF");

Evaluate

In [ ]:
y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    pass
    model = AutoReg(history, lags=best_p).fit()
    prediction = model.forecast()
    y_pred_wfv = y_pred_wfv.append(prediction)
    history = history.append(y_test[prediction.index])
    # test_mae = mean_absolute_error(y_test, y_pred_wfv)
y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()
history.tail(1)

Walk-forwad val

In [ ]:
y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    return i
y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()

#### Communicate Results

Test vs wfv predictions:

In [ ]:
df_pred_test = pd.DataFrame({"y_test": y_test, "y_pred_wfv": y_pred_wfv})
fig = px.line(df_pred_test, labels={"value": "PM2.5"})
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level",
)